In [1]:
`import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [6]:
doc = """
         드론 활용 범위도 점차 확대되고 있다. 최근에는 미세먼지 관리에 드론이 활용되고 있다.
         서울시는 '미세먼지 계절관리제' 기간인 지난달부터 오는 3월까지 4개월간 드론에 측정장치를 달아 미세먼지 집중 관리를 실시하고 있다.
         드론은 산업단지와 사업장 밀집지역을 날아다니며 미세먼지 배출 수치를 점검하고, 현장 모습을 영상으로 담는다.
         영상을 통해 미세먼지 방지 시설을 제대로 가동하지 않는 업체와 무허가 시설에 대한 단속이 한층 수월해질 전망이다.
         드론 활용에 가장 적극적인 소방청은 광범위하고 복합적인 재난 대응 차원에서 드론과 관련 전문인력 보강을 꾸준히 이어가고 있다.
         지난해 말 기준 소방청이 보유한 드론은 총 304대, 드론 조종 자격증을 갖춘 소방대원의 경우 1,860명이다.
         이 중 실기평가지도 자격증까지 갖춘 ‘드론 전문가’ 21명도 배치돼 있다.
         소방청 관계자는 "소방드론은 재난현장에서 영상정보를 수집, 산악ㆍ수난 사고 시 인명수색·구조활동,
         유독가스·폭발사고 시 대원안전 확보 등에 활용된다"며
         "향후 화재진압, 인명구조 등에도 드론을 활용하기 위해 연구개발(R&D)을 하고 있다"고 말했다.
      """

In [9]:
okt = Okt()

tokenized_doc = okt.pos(doc)
print(tokenized_doc)
tokenized_doc = ' '.join([tuple[0] for tuple in tokenized_doc if tuple[1] == 'Noun'])

print(tokenized_doc)

[('\n         ', 'Foreign'), ('드론', 'Noun'), ('활용', 'Noun'), ('범위', 'Noun'), ('도', 'Josa'), ('점차', 'Noun'), ('확대', 'Noun'), ('되고', 'Verb'), ('있다', 'Adjective'), ('.', 'Punctuation'), ('최근', 'Noun'), ('에는', 'Josa'), ('미세먼지', 'Noun'), ('관리', 'Noun'), ('에', 'Josa'), ('드론', 'Noun'), ('이', 'Josa'), ('활용', 'Noun'), ('되고', 'Verb'), ('있다', 'Adjective'), ('.', 'Punctuation'), ('\n         ', 'Foreign'), ('서울시', 'Noun'), ('는', 'Josa'), ("'", 'Punctuation'), ('미세먼지', 'Noun'), ('계절', 'Noun'), ('관리제', 'Noun'), ("'", 'Punctuation'), ('기간', 'Noun'), ('인', 'Josa'), ('지난달', 'Noun'), ('부터', 'Josa'), ('오는', 'Verb'), ('3월', 'Number'), ('까지', 'Foreign'), ('4', 'Number'), ('개', 'Noun'), ('월간', 'Noun'), ('드론', 'Noun'), ('에', 'Josa'), ('측정', 'Noun'), ('장치', 'Noun'), ('를', 'Josa'), ('달', 'Noun'), ('아', 'Josa'), ('미세먼지', 'Noun'), ('집중', 'Noun'), ('관리', 'Noun'), ('를', 'Josa'), ('실시', 'Noun'), ('하고', 'Josa'), ('있다', 'Adjective'), ('.', 'Punctuation'), ('\n         ', 'Foreign'), ('드론', 'Noun'), ('은', 'Josa'), ('산

In [10]:
n_gram_range = (2,3)

# Extract candidate words/phrases
cv = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_doc])
candidates = cv.get_feature_names_out()

print(candidates)
print(len(candidates))

['가동 업체' '가동 업체 무허가' '가장 적극' '가장 적극 소방청' '경우 실기' '경우 실기 평가' '계절 관리제'
 '계절 관리제 기간' '관계자 소방' '관계자 소방 드론' '관련 전문' '관련 전문 인력' '관리 드론' '관리 드론 활용'
 '관리 실시' '관리 실시 드론' '관리제 기간' '관리제 기간 지난달' '구조 활동' '구조 활동 유독가스' '기간 지난달'
 '기간 지난달 월간' '기준 소방청' '기준 소방청 보유' '단속 한층' '단속 한층 전망' '단지 사업' '단지 사업 밀집'
 '대원 안전' '대원 안전 확보' '대응 차원' '대응 차원 드론' '대한 단속' '대한 단속 한층' '드론 관련'
 '드론 관련 전문' '드론 드론' '드론 드론 조종' '드론 산업' '드론 산업 단지' '드론 재난' '드론 재난 현장'
 '드론 전문가' '드론 전문가 명도' '드론 조종' '드론 조종 자격증' '드론 측정' '드론 측정 장치' '드론 활용'
 '드론 활용 가장' '드론 활용 범위' '드론 활용 서울시' '드론 활용 위해' '명도 배치' '명도 배치 소방청' '모습 영상'
 '모습 영상 영상' '무허가 시설' '무허가 시설 대한' '미세먼지 계절' '미세먼지 계절 관리제' '미세먼지 관리'
 '미세먼지 관리 드론' '미세먼지 방지' '미세먼지 방지 시설' '미세먼지 배출' '미세먼지 배출 수치' '미세먼지 집중'
 '미세먼지 집중 관리' '밀집 지역' '밀집 지역 미세먼지' '방지 시설' '방지 시설 제대로' '배출 수치' '배출 수치 점검'
 '배치 소방청' '배치 소방청 관계자' '범위 점차' '범위 점차 확대' '보강 어가' '보강 어가 지난해' '보유 드론'
 '보유 드론 드론' '복합 재난' '복합 재난 대응' '사고 인명' '사고 인명 수색' '사업 밀집' '사업 밀집 지역'
 '산악 수난' '산악 수난 사고' '산업 단지' '산업 단지 사업' '서울시 미세먼지' '서울시 미세먼지 계절' '소방 드론'
 '소방 드론 재난' '소방

In [15]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)
distances = cosine_similarity(doc_embedding, candidate_embeddings)

print('document_embedding:', doc_embedding.shape)
print('candidate_embeddings:', candidate_embeddings.shape)
print('distances:', distances.shape)

document_embedding: (1, 768)
candidate_embeddings: (222, 768)
distances: (1, 222)


# 1. Basic Key BERT

In [12]:
# pick top_n words closest to the doc.
top_n = 5

keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]  # return top n keywords
print(keywords)

['드론 산업', '드론 드론 조종', '실시 드론 산업', '관리 드론 활용', '미세먼지 관리 드론']


# 2. Max Sum Similarity

In [14]:
candidates_num = 10

dist_doc_word = cosine_similarity(doc_embedding, candidate_embeddings) # (1, 72)
dist_word_word = cosine_similarity(candidate_embeddings, candidate_embeddings) # (72, 72)

words_idx = list(np.argsort(dist_doc_word)[0][-candidates_num:])
words = [candidates[idx] for idx in words_idx]

dist_word_word = dist_word_word[np.ix_(words_idx, words_idx)]

min_sim = np.inf
candidate = None

# find the least similar words
for combination in itertools.combinations(range(len(words_idx)), top_n):
    sim = sum([dist_word_word[i][j] for i in combination for j in combination if i != j])
    if sim < min_sim:
        candidate = combination
        min_sim = sim
        
result = [words[idx] for idx in candidate]

print(result)

['드론 산업 단지', '전망 드론 활용', '드론 산업', '관리 드론 활용', '미세먼지 관리 드론']


### np.ix_
 - construct an open mesh from multiple sequences.
```python
a = np.array([[1,2,3,4,5,6,7,8,9,10],[11,12,13,14,15,16,17,18,19,20],[21,22,23,24,25,26,27,28,29,30]])
_a = a[np.ix_([0,2,1],[0,1,2])]
print(_a) 
"""
[[ 1  2  3]
 [21 22 23]
 [11 12 13]]
"""
```

### itertools.combinations
 - return r length subsequences of elements from the input iterable.
 - combinations('ABCD', 2) --> AB AC AD BC BD CD
 - combinations(range(4), 3) --> 012 013 023 123
```python
for combination in itertools.combinations(range(5), 3):
  print(combination)
"""
(0, 1, 2)
(0, 1, 3)
(0, 1, 4)
(0, 2, 3)
(0, 2, 4)
(0, 3, 4)
(1, 2, 3)
(1, 2, 4)
(1, 3, 4)
(2, 3, 4)
"""
```

# 3. Maximal Marginal Relevance

In [16]:
diversity = 0.2

dist_word_doc = cosine_similarity(candidate_embeddings, doc_embedding) # (72, 1)
dist_word_word_2 = cosine_similarity(candidate_embeddings) # (72, 72)

most_similar_keyword = [np.argmax(dist_word_doc)] # [2]

# cand_indexes of words except most_similar_keyword
cand_indexes = [i for i in range(len(candidates)) if i not in most_similar_keyword] 

for _ in range(top_n-1):
    candidate_similarities = dist_word_doc[cand_indexes]
    target_similarities = np.max(dist_word_word_2[cand_indexes][:, most_similar_keyword], axis=1)
    # print(candidate_similarities.shape)
    # print(target_similarities.reshape(-1,1).shape)
    
    mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1,1)
    mmr_idx = cand_indexes[np.argmax(mmr)]
    
    most_similar_keyword.append(mmr_idx)
    cand_indexes.remove(mmr_idx)
    
result = [candidates[idx] for idx in most_similar_keyword]
print(result)

['미세먼지 관리 드론', '실시 드론 산업', '관리 드론 활용', '월간 드론 측정', '전망 드론 활용']


### np.array
```python
a = np.array([1,2,3,4,5,6,7,8,9])
a = a.reshape(3,3)
print(a)
"""
[[1 2 3]
 [4 5 6]
 [7 8 9]]
"""
print(a[[0,1,2]][:,[2]])
"""
[[3]
 [6]
 [9]]
"""
```